In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
Вниемание Не является верным решение так как не верно сделано но похоже на правду


In [60]:
data = pd.read_csv('../csv_file/train.csv')
test_data = pd.read_csv("../csv_file/valid.csv")
data.head()

C:\Users\josha\AppData\Local\Temp\ipykernel_9864\629022312.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,1447,1448,1449,1450,

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164


In [61]:
print("Train data info:")
data.info()
print("Test data info:")
test_data.info()

Train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 2666 entries, report_date to col2663
dtypes: float64(2316), int64(3), object(347)
memory usage: 294.0+ MB
Test data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642 entries, 0 to 3641
Columns: 2666 entries, report_date to col2663
dtypes: float64(2329), int64(3), object(334)
memory usage: 74.1+ MB


In [62]:
numeric_columns = data.select_dtypes(include=['number']).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].median())
test_numeric_columns = test_data.select_dtypes(include=['number']).columns
test_data[test_numeric_columns] = test_data[test_numeric_columns].fillna(test_data[test_numeric_columns].median())
threshold = 0.7 
missing_ratio = data.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > threshold].index
print(f"Columns to drop due to missing values: {columns_to_drop}")
data.drop(columns=columns_to_drop, axis=1, inplace=True)
test_data.drop(columns=columns_to_drop, axis=1, inplace=True)

Columns to drop due to missing values: Index(['col49', 'col50', 'col51', 'col52', 'col53', 'col54', 'col55', 'col56',
       'col57', 'col58',
       ...
       'col2269', 'col2270', 'col2535', 'col2536', 'col2537', 'col2538',
       'col2539', 'col2540', 'col2541', 'col2542'],
      dtype='object', length=426)


In [63]:
X = data.drop(columns=['target', 'client_id', 'report_date'], axis=1)
y = data['target']
X_test = test_data.drop(columns=['client_id', 'report_date'], axis=1)
non_numeric_columns = X.select_dtypes(exclude=['number']).columns
if len(non_numeric_columns) > 0:
    print(f"Encoding non-numeric columns: {non_numeric_columns}")
    X = pd.get_dummies(X, columns=non_numeric_columns)
    X_test = pd.get_dummies(X_test, columns=non_numeric_columns)
    X_test = X_test.reindex(columns=X.columns, fill_value=0)

Encoding non-numeric columns: Index(['col1454'], dtype='object')


In [64]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred)
print(f"ROC-AUC на валидационных данных: {roc_auc}")

ROC-AUC на валидационных данных: 0.9726610250185506


In [66]:
test_data['score'] = model.predict_proba(X_test)[:, 1]

C:\Users\josha\AppData\Local\Temp\ipykernel_9864\1435284687.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['score'] = model.predict_proba(X_test)[:, 1]


In [67]:
test_data[['client_id', 'score']].to_csv('submission_file.csv', index=False)

print("Модель успешно построена и результаты сохранены в submission_file.csv")

Модель успешно построена и результаты сохранены в submission_file.csv


In [68]:
print(test_data[['client_id','col2663']])

      client_id   col2663
0             2  0.256919
1             4  0.267221
2             9  0.257217
3            12  0.256261
4            14  0.257348
...         ...       ...
3637       4366  0.256261
3638       8298  0.256261
3639        835  0.256919
3640       8300  0.257217
3641       1974  0.285755

[3642 rows x 2 columns]


In [ ]:
from sklearn.metrics import accuracy_score
y_pred_class = model.predick(X_val)
accuracy = accuracy_score(y_val, y_pred_class)
y_test_pred_class = model.predict(X_test) 
test_accuracy = accuracy_score(test_data['target'], y_test_pred_class)
print(f"Точность на тестовых данных: {test_accuracy}")


AttributeError: 'RandomForestClassifier' object has no attribute 'predick'